In [1]:
from torchvision.models import resnet50
import torch.nn as nn
import torch as ch

In [2]:
class MutiHeadModel(ch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        num_features = model.fc.out_features
        
        self.regression = nn.Sequential(
                nn.Linear(num_features, 1),
                nn.Sigmoid()
            )

    def forward(self, x):
        clf_out = self.model(x)
        reg_out = self.regression(clf_out)

        return clf_out, reg_out

model = resnet50(weights=None)
multi_head_model = MutiHeadModel(model)

batch_size = 20
image_size = 256
num_channels = 3  # RGB images

# Create random input data
input_data = ch.randn(batch_size, num_channels, image_size, image_size)

clf_out, reg_out = multi_head_model(input_data)

# Check the output dimensions
assert clf_out.shape == (batch_size, 1000), f"Classification output shape is incorrect: {clf_out.shape}"
assert reg_out.shape == (batch_size, 1), f"Regression output shape is incorrect: {reg_out.shape}"

In [3]:
from ffcv.loader import Loader, OrderOption
from ffcv.transforms import (
    ToTensor,
    ToDevice,
    Squeeze,
    NormalizeImage,
    RandomHorizontalFlip,
    ToTorchImage,
    Convert
)
from ffcv.fields.rgb_image import (
    CenterCropRGBImageDecoder,
    RandomResizedCropRGBImageDecoder,
)
from ffcv.fields.basics import IntDecoder, FloatDecoder

import torch as ch

import numpy as np
import time

from tqdm import tqdm

print("everything is loaded completely")

train_dataset = "/home/soroush1/projects/def-kohitij/soroush1/training_fast_publish_faster/data/lamem_train_256.ffcv"
num_workers = 1
batch_size = 20
distributed = 0
in_memory = True
this_device = "cuda:0"

LAMEM_MEAN = np.load(
    "/home/soroush1/projects/def-kohitij/soroush1/pretrain-imagenet/datasets/LaMem/support_files/image_mean.npy"
)

res = 256

decoder = RandomResizedCropRGBImageDecoder((res, res))
image_pipeline = [
    decoder,
    RandomHorizontalFlip(),
    ToTensor(),
    ToDevice(ch.device(this_device), non_blocking=True),
    ToTorchImage(),
    Convert(ch.float16)
    # NormalizeImage(IMAGENET_MEAN, IMAGENET_STD, np.float16),
]

label_pipeline = [
    FloatDecoder(),
    ToTensor(),
    Squeeze(),
    ToDevice(ch.device(this_device), non_blocking=True),
]

order = OrderOption.RANDOM if distributed else OrderOption.QUASI_RANDOM

loader = Loader(
    train_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    order=order,
    os_cache=in_memory,
    drop_last=True,
    pipelines={"image": image_pipeline, "label": label_pipeline},
    distributed=distributed,
)


multi_head_model.to(this_device)
for i, (ims, labs) in tqdm(enumerate(loader)):
    print(f"{i}: {labs = }")
    print(f"{i}: {ims.size() = }")
    print(f"{i}: {ims.float().mean() = }")

    # with autocast():
    #     clf, reg = multi_head_model(ims)

    # print(f"{clf.size() = }")
    # print(f"{reg.size() = }")
    
    if i == 5:
        break

everything is loaded completely


1it [00:04,  4.37s/it]

0: labs = tensor([0.9211, 0.8684, 0.9535, 0.9487, 0.7778, 0.7632, 0.5909, 0.6000, 0.8333,
        0.7660, 0.9535, 0.7750, 0.7429, 0.8936, 0.9167, 0.4750, 0.9143, 0.7750,
        0.6176, 0.7907], device='cuda:0', dtype=torch.float64)
0: ims.size() = torch.Size([20, 3, 256, 256])
0: ims.float().mean() = tensor(115.7043, device='cuda:0')


2it [00:04,  2.08s/it]

1: labs = tensor([0.6809, 0.8333, 0.8333, 0.8667, 0.8500, 0.7778, 0.6341, 0.6818, 0.8462,
        0.7500, 0.8421, 0.7727, 0.6250, 0.8235, 0.8462, 0.7179, 0.8600, 0.8000,
        0.4634, 0.8000], device='cuda:0', dtype=torch.float64)
1: ims.size() = torch.Size([20, 3, 256, 256])
1: ims.float().mean() = tensor(111.5663, device='cuda:0')


...

3: labs = tensor([0.8421, 0.8261, 0.7292, 0.7619, 0.4615, 0.9394, 0.8684, 0.7619, 0.4545,
        0.9355, 0.7551, 0.5122, 0.7297, 0.9302, 0.5526, 0.8462, 0.8611, 0.7353,
        0.8140, 0.6154], device='cuda:0', dtype=torch.float64)
3: ims.size() = torch.Size([20, 3, 256, 256])
3: ims.float().mean() = tensor(108.0992, device='cuda:0')


5it [00:06,  1.29it/s]

4: labs = tensor([0.7959, 0.9429, 0.7556, 0.6757, 0.6429, 0.8421, 0.8718, 0.7647, 0.8049,
        0.9024, 0.8250, 0.6750, 0.7353, 0.6053, 0.8462, 0.7632, 0.6429, 0.8529,
        0.7568, 0.7692], device='cuda:0', dtype=torch.float64)
4: ims.size() = torch.Size([20, 3, 256, 256])
4: ims.float().mean() = tensor(108.2030, device='cuda:0')


5it [00:06,  1.30s/it]

5: labs = tensor([0.9118, 0.8125, 0.7000, 0.6667, 0.6389, 0.8444, 0.9474, 0.7600, 0.5641,
        0.9000, 0.8889, 0.7500, 0.9111, 0.6250, 0.8333, 0.6279, 0.8205, 0.8250,
        0.9118, 0.8919], device='cuda:0', dtype=torch.float64)
5: ims.size() = torch.Size([20, 3, 256, 256])
5: ims.float().mean() = tensor(107.0769, device='cuda:0')


In [5]:
from ffcv.loader import Loader, OrderOption
from ffcv.transforms import (
    ToTensor,
    ToDevice,
    Squeeze,
    NormalizeImage,
    RandomHorizontalFlip,
    ToTorchImage,
Convert
)
from ffcv.fields.rgb_image import (
    CenterCropRGBImageDecoder,
    RandomResizedCropRGBImageDecoder,
)
from ffcv.fields.basics import IntDecoder, FloatDecoder

import torch as ch

import numpy as np
import time

from tqdm import tqdm
from torch.cuda.amp import autocast

print("everything is loaded completely")

train_dataset = "/home/soroush1/projects/def-kohitij/soroush1/training_fast_publish_faster/data/imagenet_train_256.ffcv"
num_workers = 1
batch_size = 20
distributed = 0
in_memory = True
this_device = "cuda:0"

IMAGENET_MEAN = np.array([0.485, 0.456, 0.406]) * 255
IMAGENET_STD = np.array([0.229, 0.224, 0.225]) * 255
DEFAULT_CROP_RATIO = 224 / 256

res = 256

decoder = RandomResizedCropRGBImageDecoder((res, res))
image_pipeline = [
    decoder,
    RandomHorizontalFlip(),
    ToTensor(),
    ToDevice(ch.device(this_device), non_blocking=True),
    ToTorchImage(),
    Convert(ch.float16)
    # NormalizeImage(IMAGENET_MEAN, IMAGENET_STD, np.float16),
]

label_pipeline = [
    IntDecoder(),
    ToTensor(),
    Squeeze(),
    ToDevice(ch.device(this_device), non_blocking=True),
]

order = OrderOption.RANDOM if distributed else OrderOption.QUASI_RANDOM

loader = Loader(
    train_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    order=order,
    os_cache=in_memory,
    drop_last=True,
    pipelines={"image": image_pipeline, "label": label_pipeline},
    distributed=distributed,
)


# multi_head_model.to(this_device)
for i, (ims, labs) in tqdm(enumerate(loader)):
    print(f"{i}: {labs = }")
    print(f"{i}: {ims.size() = }")
    print(f"{i}: {ims.mean() = }")
    
    # # with autocast():
    # clf, reg = multi_head_model(ims)

    # print(f"{clf.size() = }")
    # print(f"{reg.size() = }")
    
    if i == 5:
        break

everything is loaded completely


5it [00:02,  1.68it/s]

0: labs = tensor([395, 718, 241, 356, 671, 312,  66, 656, 396, 462, 733, 925, 242, 851,
        670, 357, 308, 734, 356, 913], device='cuda:0')
0: ims.size() = torch.Size([20, 3, 256, 256])
0: ims.mean() = tensor(115.9375, device='cuda:0', dtype=torch.float16)
1: labs = tensor([169, 472, 318, 671, 925, 817, 241, 308, 396,  67, 105, 472, 913, 809,
        357, 632,  33, 312, 242, 549], device='cuda:0')
1: ims.size() = torch.Size([20, 3, 256, 256])
1: ims.mean() = tensor(101.4375, device='cuda:0', dtype=torch.float16)
2: labs = tensor([950, 307, 460, 460, 949, 460, 821, 671, 551, 241, 310, 105, 949, 551,
        871,  12, 851, 105, 461, 853], device='cuda:0')
2: ims.size() = torch.Size([20, 3, 256, 256])
2: ims.mean() = tensor(119.0625, device='cuda:0', dtype=torch.float16)
3: labs = tensor([851, 657, 578, 809, 549, 809, 513, 396, 103, 733, 396, 513, 396, 307,
        954, 818, 733, 170, 396, 949], device='cuda:0')
3: ims.size() = torch.Size([20, 3, 256, 256])
3: ims.mean() = tensor(127.

In [24]:
LAMEM_MEAN = np.load(
    "/home/soroush1/projects/def-kohitij/soroush1/pretrain-imagenet/datasets/LaMem/support_files/image_mean.npy"
)

In [26]:
LAMEM_MEAN.shape

(256, 256, 3)